Казакова Анастасия Сергеевна. Тестовое в Вконтакте. Идея для решения взята из данной статьи: https://developer.nvidia.com/blog/using-neural-networks-for-your-recommender-system/

In [1]:
# Загрузка необходимых библиотек
import pandas as pd
import re
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.callbacks import History 
from tensorflow.keras.callbacks import ModelCheckpoint

В ходе работы использовался google диск. Для корректной работы или необходимо загрузить датасет к себе в гугл диск или поменять значения путей файлов

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Рапаковка архива (путь нужно указать свой)
!unzip  '/content/drive/MyDrive/ml-latest.zip' -d  '/content'

Archive:  /content/drive/MyDrive/ml-latest.zip
   creating: /content/ml-latest/
  inflating: /content/ml-latest/links.csv  
  inflating: /content/ml-latest/tags.csv  
  inflating: /content/ml-latest/genome-tags.csv  
  inflating: /content/ml-latest/ratings.csv  
  inflating: /content/ml-latest/README.txt  
  inflating: /content/ml-latest/genome-scores.csv  
  inflating: /content/ml-latest/movies.csv  


# Работа с данными


In [9]:
# Считывание файла
gen_tag = pd.read_csv('/content/ml-latest/genome-tags.csv')
gen_tag.head()

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s


In [10]:
# Проверка на Nan значения
gen_tag.isnull().sum()

tagId    0
tag      0
dtype: int64

In [11]:
# Считывание файла
gen_score = pd.read_csv('/content/ml-latest/genome-scores.csv')
gen_score.head()

,movieId,tagId,relevance
0,1,1,0.02900
1,1,2,0.02375
2,1,3,0.05425
3,1,4,0.06875
4,1,5,0.16000


In [12]:
# Проверка на Nan значения
gen_score.isnull().sum()

movieId      0
tagId        0
relevance    0
dtype: int64

In [13]:
# Считывание файла
tags = pd.read_csv('/content/ml-latest/tags.csv')
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


In [14]:
# Проверка на Nan значения
tags.isnull().sum()

userId        0
movieId       0
tag          16
timestamp     0
dtype: int64

In [15]:
# Вывожу строки, в которых Nan
tags[tags['tag'].isnull() ==True]

,userId,movieId,tag,timestamp
483263,80439,123,NaN,1199450867
483264,80439,346,NaN,1199451946
483268,80439,1184,NaN,1199452261
483275,80439,1785,NaN,1199452006
483276,80439,2194,NaN,1199450677
483278,80439,2691,NaN,1199451002
483286,80439,4103,NaN,1199451920
483288,80439,4473,NaN,1199451040
483290,80439,4616,NaN,1199452441
483306,80439,7624,NaN,1199452266


In [16]:
# Удаляю Nan значения
tags.dropna(inplace=True)
tags.head()

,userId,movieId,tag,timestamp
0,14,110,epic,1443148538
1,14,110,Medieval,1443148532
2,14,260,sci-fi,1442169410
3,14,260,space action,1442169421
4,14,318,imdb top 250,1442615195


# Нереализованная работа с тэгами


In [17]:
# Объединим таблицы, чтобы хранить имя тэга и его релевантность
genome_data = gen_score.merge(gen_tag,how='left', on='tagId')
genome_data.head()

,movieId,tagId,relevance,tag
0,1,1,0.02900,007
1,1,2,0.02375,007 (series)
2,1,3,0.05425,18th century
3,1,4,0.06875,1920s
4,1,5,0.16000,1930s


In [18]:
# Проверка на Nan значения
genome_data.isnull().sum()

movieId      0
tagId        0
relevance    0
tag          0
dtype: int64

В следующих двух ячейках я хотела применить One Hot Encoding, но к сожалению из-за больших объемов данных ОЗУ не хватало и колаб обновлялся. В теории это можно было бы сделать в другом месте, а результат загрузить в блокнот.


Если бы данная работа с тэгами была проведена успешно, то можно было бы создать нейронную сеть, которая отвечает за контекст (как раз таки One Hot тэги, но в них стояли бы не 1 и 0, а их релевантность и 0)

In [19]:
#from sklearn.preprocessing import OneHotEncoder
#enc = OneHotEncoder(handle_unknown='ignore')
#tag_types = genome_data['tag'].unique()
#dum_df = pd.get_dummies(genome_data, columns=['tag'], prefix=['Type_is'] )
#dum_df

In [ ]:
#tags_names = genome_data['tag'].unique()
#for col in tags_names:
 #   genome_data[col] = genome_data['tag'].apply(lambda x: 1 if col in x else 0)
#genome_data.head()

# Работа с ссылками

In [22]:
# Считывание файла
links = pd.read_csv('/content/ml-latest/links.csv')
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [23]:
# Проверка на Nan значения
links.isnull().sum()

movieId      0
imdbId       0
tmdbId     181
dtype: int64

По идее в данном задании я решила отбросить эти данные поскольку часть из них пустая да и в целом данный файл не несет смысловой нагрузки. Только если парсерить сайты по ссылке (возможно помогла бы wikidata) и искать там новые фичи

# Работа с рейтингами

In [2]:
# Считывание файла
ratings = pd.read_csv('/content/ml-latest/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [3]:
# Проверка на Nan значения
ratings.isnull().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [4]:
# Проверим наличие повторов (например, пользователь поставил фильму
# одну оценку, а потом поменял её из-за разных факторов (мог пересмотреть фильм))
ratings[ratings.duplicated(['userId','movieId'])]

,userId,movieId,rating,timestamp


In [5]:
# Смотрим типы данных
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 847.0 MB


In [6]:
# Количество фильмов и пользователей с рейтингами
ratings['movieId'].nunique(), ratings['userId'].nunique()

(53889, 283228)

# Создание простой нейронной сети с эмбеддингами

In [7]:
# Для метода коллаборативной фильтрации выделяем таблицу
collab_filt = ratings[['userId','movieId','rating']]
collab_filt.head()

,userId,movieId,rating
0,1,307,3.5
1,1,481,3.5
2,1,1091,1.5
3,1,1257,4.5
4,1,1449,4.5


In [8]:
# Производим нормирование рейтингов
scaler = MinMaxScaler()
collab_filt['rating'] = scaler.fit_transform(collab_filt['rating'].values.reshape(-1,1))
collab_filt.head()

<ipython-input-8-b6901712efd4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  collab_filt['rating'] = scaler.fit_transform(collab_filt['rating'].values.reshape(-1,1))


,userId,movieId,rating
0,1,307,0.666667
1,1,481,0.666667
2,1,1091,0.222222
3,1,1257,0.888889
4,1,1449,0.888889


In [9]:
# Сохранение файла на случай нехватки ОЗУ (путь нужно указать свой)
collab_filt.to_parquet('/content/drive/MyDrive/collab_filt.parquet')

In [2]:
# Считывание данных (путь нужно указать свой)
collab_filt = pd.read_parquet('/content/drive/MyDrive/collab_filt.parquet')

Теперь назревает вопрос: как делить на train и test выборку, ведь может быть ситуация, в которой при неправильном делении несколько пользователей полностью уйдут в test не побывав в train. Аналогично с фильмами. Поэтому я провела небольшое исследование и выяснила, что при группировке по пользователям и разбивания train датасета так, чтобы в нём было 80% записей из каждого id пользователя мы получаем 51623 из 53889 фильмов в train выборке. При этом если наоборот, группировать по фильму, то мы получим 281933 из 283228 пользователей в train. Поэтому я решила оставить тот вариант, где в процентном соотношении мы теряем меньше данных

In [5]:
print(51623/53889*100, 281933/283228*100)

95.79506021637069 99.54277119493835


Так как вторая переменная больше, это значит, что при группировке по фильмам мы также заберем 99% пользователей в train выборку. Поэтому я решила разделять данные именно так

In [3]:
# Разделение данных на train выборку (80% записей из каждого фильма)
train = collab_filt.groupby(['movieId'], group_keys=False).apply(lambda x: x.sample(frac=0.8))
train.head()

,userId,movieId,rating
23342360,238662,1,0.777778
20252868,206495,1,0.777778
25784304,263327,1,1.000000
15195870,155267,1,0.888889
18157876,185447,1,0.666667


In [26]:
# проведение мини исследования, описанного выше
#all_moviesId = collab_filt['movieId'].unique()
#all_userID = collab_filt['userId'].unique()
#all_ratings = collab_filt['rating'].unique()
#train_moviesId = train['movieId'].unique()
#train_userID = train['userId'].unique()
#train_ratings = train['rating'].unique()
#print(len(all_moviesId), len(train_moviesId), len(all_userID), len(train_userID), len(all_ratings), len(train_ratings))

In [4]:
# Для создания test выборки просто из изначальных данных убираем train
test = collab_filt.drop(index = train.index)
test.head()

,userId,movieId,rating
0,1,307,0.666667
3,1,1257,0.888889
4,1,1449,0.888889
7,1,2134,0.888889
11,1,3020,0.777778


In [28]:
# Вывод доступных девайсов
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 9984167313016003576
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14343274496
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4596614773285059696
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
xla_global_id: 416903419
]


Данную задачу я рассматриваю в виде задачи регрессии, поскольку считаю, что важно быть "около" правильного ответа, что поможет при ранжировании сохранить порядок. При задаче классификации всё может выглядеть "грубее" и будет не понятно что первым рекомендовать из всех элементов одного класса. Поэтому  loss будет MAE, а метрика MAPE

In [5]:
# Поскольку у меня ограничены возможности техники, размер эмбеддингов я беру
# равным 12 (при бОльшем размере выходит ошибка памяти)
embeddings_size = 12
usr, mv = collab_filt.shape[0], collab_filt.shape[1]

# Считываю входные данные (id пользователя и id фильма)
xusers_in = layers.Input(name="xusers_in", shape=(1,))
xmovies_in = layers.Input(name="xmovies_in", shape=(1,))

# Далее описывается первый блок, который состоит из эмбеддингов пользователя
# и фильма с операцией dot
# В начеле составляем эмбеддинг и изменяем размер для пользователя
cf_xusers_emb = layers.Embedding(name="cf_xusers_emb", input_dim=usr, output_dim=embeddings_size)(xusers_in)
cf_xusers = layers.Reshape(name='cf_xusers', target_shape=(embeddings_size,))(cf_xusers_emb)
# Теперь проделываем то же самое с фильмами
cf_xmovies_emb = layers.Embedding(name="cf_xmovies_emb", input_dim=mv, output_dim=embeddings_size)(xmovies_in)
cf_xmovies = layers.Reshape(name='cf_xmovies', target_shape=(embeddings_size,))(cf_xmovies_emb)
# Производим операцию Dot
cf_xx = layers.Dot(name='cf_xx', normalize=True, axes=1)([cf_xusers, cf_xmovies])

# Ниже приведем второй блок, который является полносвязным
# Также переводим в эмбеддинги и меняем размер для пользователей
nn_xusers_emb = layers.Embedding(name="nn_xusers_emb", input_dim=usr, output_dim=embeddings_size)(xusers_in)
nn_xusers = layers.Reshape(name='nn_xusers', target_shape=(embeddings_size,))(nn_xusers_emb)
# Аналогично для фильмов
nn_xmovies_emb = layers.Embedding(name="nn_xmovies_emb", input_dim=mv, output_dim=embeddings_size)(xmovies_in)
nn_xmovies = layers.Reshape(name='nn_xmovies', target_shape=(embeddings_size,))(nn_xmovies_emb)
# Соединяем данные пользователей и фильмов и применяем полносвязный слой 
nn_xx = layers.Concatenate()([nn_xusers, nn_xmovies])
nn_xx = layers.Dense(name="nn_xx", units=int(embeddings_size/2), activation='relu')(nn_xx)

# Соединяем оба блока
y_out = layers.Concatenate()([cf_xx, nn_xx])
y_out = layers.Dense(name="y_out", units=1, activation='linear')(y_out)
# Создаем модель с заданными параметрами
model = models.Model(inputs=[xusers_in,xmovies_in], outputs=y_out, name="Neural_CollaborativeFiltering")
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_percentage_error'])

По хорошему, данную сеть необходимо изучать больше,чем 5 эпохах. Но из-за сжатых сроков я обучила её на 5 эпохах

In [6]:
# Обучаем модель и каждую эпоху сохраняем её веса
EPOCHS = 5
# (путь нужно указать свой)
filepath='/content/drive/MyDrive/MAPE.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    monitor='val_mean_absolute_percentage_error',
    mode='max',
    save_best_only=True)
training = model.fit(x=[train['userId'].values, train['movieId'].values], y=train['rating'], epochs=EPOCHS, batch_size=2048, shuffle=True, verbose=1, validation_split=0.3, callbacks=[model_checkpoint_callback])
model = training.model

Epoch 1/5
7590/7590 [==============================] - 1178s 154ms/step - loss: 0.1773 - mean_absolute_percentage_error: 7409072.5000 - val_loss: 0.1606 - val_mean_absolute_percentage_error: 13781461.0000
Epoch 2/5
7590/7590 [==============================] - 1111s 146ms/step - loss: 0.1650 - mean_absolute_percentage_error: 7193223.5000 - val_loss: 0.1591 - val_mean_absolute_percentage_error: 13639547.0000
Epoch 3/5
7590/7590 [==============================] - 1107s 146ms/step - loss: 0.1631 - mean_absolute_percentage_error: 7091347.0000 - val_loss: 0.1587 - val_mean_absolute_percentage_error: 13657139.0000
Epoch 4/5
7590/7590 [==============================] - 1108s 146ms/step - loss: 0.1621 - mean_absolute_percentage_error: 7035304.0000 - val_loss: 0.1585 - val_mean_absolute_percentage_error: 13545010.0000
Epoch 5/5
7590/7590 [==============================] - 1112s 146ms/step - loss: 0.1616 - mean_absolute_percentage_error: 7008496.5000 - val_loss: 0.1586 - val_mean_absolute_percent

In [7]:
# Тестируем модель
test['yhat'] = model.predict([test['userId'], test['movieId']])
test.head()

173371/173371 [==============================] - 350s 2ms/step


,userId,movieId,rating,yhat
0,1,307,0.666667,0.591172
3,1,1257,0.888889,0.591172
4,1,1449,0.888889,0.591172
7,1,2134,0.888889,0.591172
11,1,3020,0.777778,0.591172


In [ ]:
# Блок на случай нехватки ОЗУ (путь нужно указать свой)
model.load_weights('/content/drive/MyDrive/MAPE.hdf5')
test['yhat'] = model.predict([test['userId'], test['movieId']])
test.head()

По началу у меня были проблемы с работой модели, она выдавала всегда по 2-3 разных значения. Поэтому в приведенной ниже ячейке я вывожу количество разных значений, которые предсказала модель

In [8]:
test['yhat'].nunique()

270720

# Рекомендации

In [9]:
# Считывание файла
movies = pd.read_csv('/content/ml-latest/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [12]:
# Вводим id пользователя для рекомендации фильма
i= input()
print('--- user', i, '---')
# В дальнейшем выдадим 5 рекомендаций
top = 5
# Чтобы не рекомендовать старые фильмы, создадим массив, который хранит в себе
# фильмы, которые пользователь не смотрел
all_moviesId = movies['movieId'].values
user_see_movie = collab_filt[collab_filt['userId']==int(i)]['movieId'].values
user_not_see_movie = np.array([item for item in list(all_moviesId) if item not in list(user_see_movie)])
user_not_see_movie= user_not_see_movie.reshape((user_not_see_movie.shape[0],1))
# Для предсказаний создадим столбец с id пользователя
# такого же размера как и непосмотренные фильмы
user_id = np.full(shape=user_not_see_movie.shape, fill_value=int(i))
predict_dataframe = pd.DataFrame(np.hstack((user_id,user_not_see_movie)),columns=['userId', 'movieId'])
# Предскажем рейтинг и выведем первые 5 id фмльмов
predict_dataframe['yhat'] = model.predict([predict_dataframe['userId'],predict_dataframe['movieId']])
predict_dataframe.sort_values('yhat',ascending=False)['movieId'].values[:top]

1
--- user 1 ---
1816/1816 [==============================] - 4s 2ms/step


array([     1, 150202, 150092, 150104, 150106])

Далее идёт блок улучшения моего решения. Мысли, как можно было бы усовершенствовать нейронную сеть и рекомендации

# Работа с фильмами и извлечение их признаков

In [69]:
# Считывание файла
movies = pd.read_csv('/content/ml-latest/movies.csv')
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [70]:
# Информация о данных
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  58098 non-null  int64 
 1   title    58098 non-null  object
 2   genres   58098 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.3+ MB


In [71]:
# Проверка на Nan значения
movies.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

In [72]:
# Извлечение названия фильма и года
movies['new_title']=movies['title'].apply(lambda x: re.sub('[\(\[].*?[\)\]]', '', x).strip())
movies['movie_year'] = movies['title'].apply(lambda x: x.split('(')[-1].replace(')',''.strip()) 
if '(' in x else np.nan)

In [73]:
# One Hot Encoding жанров
tags = [i.split('|') for i in movies['genres'].unique()]
columns = list(set([i for lst in tags for i in lst]))
for col in columns:
    movies[col] = movies['genres'].apply(lambda x: 1 if col in x else 0)

In [74]:
# Удаляем старое название и жанр
drop_movies = movies.drop(columns=['title','genres'], axis=1)
drop_movies

,movieId,new_title,movie_year,Children,Horror,Adventure,Film-Noir,Drama,Documentary,Sci-Fi,...,IMAX,Comedy,Action,War,Fantasy,Musical,Crime,(no genres listed),Thriller,Animation
0,1,Toy Story,1995,1,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
1,2,Jumanji,1995,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men,1995,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,1995,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,1995,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58093,193876,The Great Glinka,1946,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
58094,193878,Les tribulations d'une caissière,2011,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
58095,193880,Her Name Was Mumu,2016,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
58096,193882,Flora,2017,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [75]:
# Выводим уникальные значения годов
drop_movies['movie_year'].unique()

array(['1995', '1994', '1996', '1976', '1992', '1988', '1967', '1993',
       '1964', '1977', '1965', '1982', '1985', '1990', '1991', '1989',
       '1937', '1940', '1969', '1981', '1973', '1970', '1960', '1955',
       '1959', '1968', '1980', '1975', '1986', '1948', '1943', '1950',
       '1946', '1987', '1997', '1974', '1956', '1958', '1949', '1972',
       '1998', '1933', '1952', '1951', '1957', '1961', '1954', '1934',
       '1944', '1963', '1942', '1941', '1953', '1939', '1947', '1945',
       '1938', '1935', '1936', '1926', '1932', '1979', '1971', '1978',
       '1966', '1962', '1983', '1984', '1931', '1922', '1999', '1927',
       '1929', '1930', '1928', '1925', '1914', '2000', '1919', '1923',
       '1920', '1918', '1921', '2001', '1924', '2002', '2003', '1915',
       '2004', '1916', '1917', '1948 ', '1965 ', '1988 ', '1999 ', '2005',
       '2006', '2003 ', '2002 ', '1995 ', '1902', nan, '2001 ', '1989 ',
       '1971 ', '1903', '2007', '2006 ', '2008', '1980 ', '2008 ',
    

In [76]:
# Как можно заметить, видимо в названии некоторых фильмов в скобках
# стоял не год, а что-то другое. Поэтому нужно выловить все неправильные
# значения и поменять их на nan (если таких значений было бы много, то лучше
# усовершенствовать решение функцией isin)
wrong_index = drop_movies[(drop_movies['movie_year']=='Bicicleta, cullera, poma') | (drop_movies['movie_year']=='Das Millionenspiel') | (drop_movies['movie_year']=='Your Past Is Showing') | (drop_movies['movie_year']=='Close Relations') | (drop_movies['movie_year']=='2006–2007')].index
wrong_index

Int64Index([15719, 17444, 35140, 45567, 48291], dtype='int64')

In [77]:
# Замента на nan
drop_movies['movie_year'].loc[wrong_index] = np.nan

<ipython-input-77-0bfa69282c34>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_movies['movie_year'].loc[wrong_index] = np.nan


In [78]:
# Но также у нас есть значение с дефисом. Так как оно одно, то можно убрать дефис
drop_movies['movie_year'].loc[drop_movies[drop_movies['movie_year']=='2009– '].index] = '2009'

<ipython-input-78-e1eb9590c6c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drop_movies['movie_year'].loc[drop_movies[drop_movies['movie_year']=='2009– '].index] = '2009'


In [79]:
# Заменяем nan на медианное значение в выборке и переводим года в int
drop_movies['movie_year'] = drop_movies['movie_year'].fillna(drop_movies['movie_year'].median())
drop_movies['movie_year'] = drop_movies['movie_year'].astype('int')

In [80]:
# Используем нормализацию данных, чтобы значения были от 0 до 1
scaler = MinMaxScaler()
drop_movies['movie_year'] = scaler.fit_transform(drop_movies['movie_year'].values.reshape(-1,1))
drop_movies

,movieId,new_title,movie_year,Children,Horror,Adventure,Film-Noir,Drama,Documentary,Sci-Fi,...,IMAX,Comedy,Action,War,Fantasy,Musical,Crime,(no genres listed),Thriller,Animation
0,1,Toy Story,0.840278,1,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
1,2,Jumanji,0.840278,1,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,Grumpier Old Men,0.840278,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4,Waiting to Exhale,0.840278,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,Father of the Bride Part II,0.840278,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58093,193876,The Great Glinka,0.500000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
58094,193878,Les tribulations d'une caissière,0.951389,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
58095,193880,Her Name Was Mumu,0.986111,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
58096,193882,Flora,0.993056,0,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [81]:
# Удаляем лишние колонки
movie_features_data = drop_movies.drop(['new_title'],axis=1)
movie_features_data

,movieId,movie_year,Children,Horror,Adventure,Film-Noir,Drama,Documentary,Sci-Fi,Mystery,...,IMAX,Comedy,Action,War,Fantasy,Musical,Crime,(no genres listed),Thriller,Animation
0,1,0.840278,1,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
1,2,0.840278,1,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,3,0.840278,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,4,0.840278,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,5,0.840278,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58093,193876,0.500000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
58094,193878,0.951389,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
58095,193880,0.986111,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
58096,193882,0.993056,0,1,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [82]:
# Поскольку мне не хватало ОЗУ, то я скачивала получившийся файл для дальнейшей работы
# (путь нужно указать свой)
movie_features_data.to_parquet('/content/drive/MyDrive/movie_features_data.parquet')

# Создание новых фичей контекста и фильма

In [3]:
# Созданим новые фичи контекста. Для этого временную метку выставления рейтинга
# переведем в формат даты
ratings['timestamp'] = ratings['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
ratings['just_date'] = ratings['timestamp'].dt.date
ratings.head()

,userId,movieId,rating,timestamp,just_date
0,1,307,3.5,2009-10-27 21:00:21,2009-10-27
1,1,481,3.5,2009-10-27 21:04:16,2009-10-27
2,1,1091,1.5,2009-10-27 21:04:31,2009-10-27
3,1,1257,4.5,2009-10-27 21:04:20,2009-10-27
4,1,1449,4.5,2009-10-27 21:01:04,2009-10-27


In [4]:
# Из полученной даты извлечем день, месяц и год
ratings['day'] = ratings['timestamp'].dt.day
ratings['month'] = ratings['timestamp'].dt.month
ratings['year'] = ratings['timestamp'].dt.year
ratings.head()

,userId,movieId,rating,timestamp,just_date,day,month,year
0,1,307,3.5,2009-10-27 21:00:21,2009-10-27,27,10,2009
1,1,481,3.5,2009-10-27 21:04:16,2009-10-27,27,10,2009
2,1,1091,1.5,2009-10-27 21:04:31,2009-10-27,27,10,2009
3,1,1257,4.5,2009-10-27 21:04:20,2009-10-27,27,10,2009
4,1,1449,4.5,2009-10-27 21:01:04,2009-10-27,27,10,2009


In [5]:
# Также можно подсчитать количество пользователей, которые 
# поставили каждому фильму оценку
ratings['user_count'] = ratings.groupby('movieId')['userId'].transform('nunique')
ratings.head()

,userId,movieId,rating,timestamp,just_date,day,month,year,user_count
0,1,307,3.5,2009-10-27 21:00:21,2009-10-27,27,10,2009,7958
1,1,481,3.5,2009-10-27 21:04:16,2009-10-27,27,10,2009,6037
2,1,1091,1.5,2009-10-27 21:04:31,2009-10-27,27,10,2009,6138
3,1,1257,4.5,2009-10-27 21:04:20,2009-10-27,27,10,2009,5902
4,1,1449,4.5,2009-10-27 21:01:04,2009-10-27,27,10,2009,6867


In [6]:
# На всякий случай проверим сходятся ли данные
ratings[ratings['movieId']==307]

,userId,movieId,rating,timestamp,just_date,day,month,year,user_count
0,1,307,3.5,2009-10-27 21:00:21,2009-10-27,27,10,2009,7958
870,6,307,4.0,1996-05-14 07:34:08,1996-05-14,14,5,1996,7958
4787,56,307,4.0,2013-11-05 04:28:48,2013-11-05,5,11,2013,7958
6114,71,307,5.0,2009-11-09 19:36:54,2009-11-09,9,11,2009,7958
8909,84,307,3.0,2001-08-29 03:25:19,2001-08-29,29,8,2001,7958
...,...,...,...,...,...,...,...,...,...
27724043,282891,307,5.0,1999-10-02 05:51:59,1999-10-02,2,10,1999,7958
27724319,282898,307,5.0,2012-05-20 21:22:45,2012-05-20,20,5,2012,7958
27733080,283000,307,4.0,2014-11-08 23:32:01,2014-11-08,8,11,2014,7958
27743374,283116,307,5.0,2000-02-17 17:11:28,2000-02-17,17,2,2000,7958


In [7]:
# Также добавим средний рейтинг для каждого фильма
ratings['mean_rating'] = ratings.groupby('movieId')['rating'].transform(lambda x : x.mean())
ratings.head()

,userId,movieId,rating,timestamp,just_date,day,month,year,user_count,mean_rating
0,1,307,3.5,2009-10-27 21:00:21,2009-10-27,27,10,2009,7958,3.971727
1,1,481,3.5,2009-10-27 21:04:16,2009-10-27,27,10,2009,6037,3.339241
2,1,1091,1.5,2009-10-27 21:04:31,2009-10-27,27,10,2009,6138,2.806207
3,1,1257,4.5,2009-10-27 21:04:20,2009-10-27,27,10,2009,5902,3.828617
4,1,1449,4.5,2009-10-27 21:01:04,2009-10-27,27,10,2009,6867,3.918378


In [8]:
# Для проверки выведем фильмы, у которых 1 оценка
ratings[ratings['user_count'] == ratings['user_count'].min()]

,userId,movieId,rating,timestamp,just_date,day,month,year,user_count,mean_rating
29721,277,131564,3.5,2015-04-02 00:46:20,2015-04-02,2,4,2015,1,3.5
29722,277,131566,3.5,2015-04-02 00:47:39,2015-04-02,2,4,2015,1,3.5
45556,449,148416,0.5,2017-03-18 19:51:52,2017-03-18,18,3,2017,1,0.5
45645,449,171753,1.5,2017-05-05 04:28:13,2017-05-05,5,5,2017,1,1.5
47872,470,161864,0.5,2017-02-26 16:20:38,2017-02-26,26,2,2017,1,0.5
...,...,...,...,...,...,...,...,...,...,...
27680666,282403,167894,1.0,2018-04-20 17:04:45,2018-04-20,20,4,2018,1,1.0
27708795,282732,161572,3.5,2017-09-03 03:07:50,2017-09-03,3,9,2017,1,3.5
27734907,283000,117857,3.5,2014-11-30 03:26:09,2014-11-30,30,11,2014,1,3.5
27734957,283000,133409,3.5,2015-05-13 17:48:51,2015-05-13,13,5,2015,1,3.5


In [9]:
# выведем любой из них чтобы проверить среднее значение
ratings[ratings['movieId']==131564]

,userId,movieId,rating,timestamp,just_date,day,month,year,user_count,mean_rating
29721,277,131564,3.5,2015-04-02 00:46:20,2015-04-02,2,4,2015,1,3.5


In [10]:
# Из-за нехватки ОЗУ сохраняем файл (путь нужно указать свой)
ratings.to_parquet('/content/drive/MyDrive/feature_ratings.parquet')

# Ещё немного изменений в данных рейтинга

In [3]:
# Считаем полученный в предыдущем блоке файл (путь нужно указать свой)
new_ratings = pd.read_parquet('/content/drive/MyDrive/feature_ratings.parquet')
new_ratings.head()

,userId,movieId,rating,timestamp,just_date,day,month,year,user_count,mean_rating
0,1,307,3.5,2009-10-27 21:00:21,2009-10-27,27,10,2009,7958,3.971727
1,1,481,3.5,2009-10-27 21:04:16,2009-10-27,27,10,2009,6037,3.339241
2,1,1091,1.5,2009-10-27 21:04:31,2009-10-27,27,10,2009,6138,2.806207
3,1,1257,4.5,2009-10-27 21:04:20,2009-10-27,27,10,2009,5902,3.828617
4,1,1449,4.5,2009-10-27 21:01:04,2009-10-27,27,10,2009,6867,3.918378


In [4]:
# Удалим ненужные столбцы
new_ratings = new_ratings.drop(['timestamp','just_date'], axis=1)
new_ratings.head()

,userId,movieId,rating,day,month,year,user_count,mean_rating
0,1,307,3.5,27,10,2009,7958,3.971727
1,1,481,3.5,27,10,2009,6037,3.339241
2,1,1091,1.5,27,10,2009,6138,2.806207
3,1,1257,4.5,27,10,2009,5902,3.828617
4,1,1449,4.5,27,10,2009,6867,3.918378


In [5]:
# Также отмасштабируем наши численные данные 
scaler = MinMaxScaler()
columns = ['rating','day','month','year', 'user_count', 'mean_rating']
new_ratings[columns] = scaler.fit_transform(new_ratings[columns])
new_ratings.head()

,userId,movieId,rating,day,month,year,user_count,mean_rating
0,1,307,0.666667,0.866667,0.818182,0.608696,0.081196,0.771495
1,1,481,0.666667,0.866667,0.818182,0.608696,0.061593,0.630943
2,1,1091,0.222222,0.866667,0.818182,0.608696,0.062624,0.512490
3,1,1257,0.888889,0.866667,0.818182,0.608696,0.060216,0.739693
4,1,1449,0.888889,0.866667,0.818182,0.608696,0.070063,0.759639


In [6]:
# Также сохраним полученные данные (путь нужно указать свой)
new_ratings.to_parquet('/content/drive/MyDrive/new_ratings.parquet')

# Считывание и объединение данных

In [ ]:
# Считывание необходимых данных (путь нужно указать свой)
new_ratings = pd.read_parquet('new_ratings.parquet')

In [ ]:
# Считывание необходимых данных (путь нужно указать свой)
movie_features_data = pd.read_parquet('movie_features_data.parquet')

Далее я хотела соединить полученные таблицы и условно разделить данные на контекстные данные (информация о рейтингах фильмов такие как день, количество голосов и т.д.), а также данные о фильме (One Hot жанр фильма). Но опять таки нехватка ОЗУ не дала мне этого сделать :(

In [ ]:
#all_features_data = new_ratings.merge(movie_features_data, on ='movieId', how = 'left')
#all_features_data.head()

# Попытка улучшить нейронную сеть с помощью контекстной информации

In [3]:
# Считываем необходимые данные (путь нужно указать свой)
new_ratings = pd.read_parquet('/content/drive/MyDrive/new_ratings.parquet')
new_ratings.head()

,userId,movieId,rating,day,month,year,user_count,mean_rating
0,1,307,0.666667,0.866667,0.818182,0.608696,0.081196,0.771495
1,1,481,0.666667,0.866667,0.818182,0.608696,0.061593,0.630943
2,1,1091,0.222222,0.866667,0.818182,0.608696,0.062624,0.512490
3,1,1257,0.888889,0.866667,0.818182,0.608696,0.060216,0.739693
4,1,1449,0.888889,0.866667,0.818182,0.608696,0.070063,0.759639


In [4]:
# Выделяем контекстные данные
context = new_ratings[['day', 'month','year', 'user_count', 'mean_rating']]

In [5]:
# Разделение данных на train выборку (80% записей из каждого фильма)
train = new_ratings.groupby(['movieId'], group_keys=False).apply(lambda x: x.sample(frac=0.8))
train.head()

,userId,movieId,rating,day,month,year,user_count,mean_rating
5945016,61080,1,1.000000,0.233333,0.454545,0.043478,0.698667,0.752589
15607557,159328,1,0.555556,0.933333,0.363636,0.086957,0.698667,0.752589
13378453,136797,1,1.000000,0.633333,0.363636,0.043478,0.698667,0.752589
17410882,177859,1,0.666667,0.766667,0.545455,0.521739,0.698667,0.752589
8326452,85775,1,0.888889,0.100000,0.909091,0.434783,0.698667,0.752589


In [6]:
# Для создания test выборки просто из изначальных данных убираем train
test = new_ratings.drop(index = train.index)
test.head()

,userId,movieId,rating,day,month,year,user_count,mean_rating
10,1,2986,0.444444,0.866667,0.818182,0.608696,0.061828,0.441016
11,1,3020,0.777778,0.866667,0.818182,0.608696,0.079410,0.688024
12,1,3424,0.888889,0.866667,0.818182,0.608696,0.074124,0.740996
13,1,3698,0.666667,0.866667,0.818182,0.608696,0.084369,0.604332
14,1,3826,0.333333,0.866667,0.818182,0.608696,0.090788,0.454696


In [7]:
# Тут я попыталась уменьшить размер данных, но все равно не получилось
#train= train.iloc[150:15000000]
#test = test.iloc[150:1500000]

In [8]:
# Разбиваем контекстные данные на train и test
train_context = train[['day', 'month','year', 'user_count', 'mean_rating']]
test_context = test[['day', 'month','year', 'user_count', 'mean_rating']]

Далее я попыталась написать нейронную сеть, которая уже будет стостоять из трех блоков. Два как раньше и один блок будет полносвязный с контекстными данными. Но к сожалению ошибка нехватки памяти снова не дала мне этого сделать

In [9]:
# Поскольку у меня ограничены возможности техники, размер эмбеддингов я беру
# равным 10 (хотя все равно даже с таким размером не получается)
embeddings_size = 10
usr, prd = new_ratings.shape[0], new_ratings.shape[1]
ctx = len(context)

# Считываю входные данные (id пользователя и id фильма)
users = layers.Input(name="users", shape=(1,))
movies = layers.Input(name="movies", shape=(1,))
# Далее описывается первый блок, который состоит из эмбеддингов пользователя
# и фильма с операцией dot
# В начеле составляем эмбеддинг и изменяем размер для пользователя
new_users = layers.Embedding(name="cf_xusers_emb", input_dim=usr, output_dim=embeddings_size)(users)
new_users = layers.Reshape(name='cf_xusers', target_shape=(embeddings_size,))(new_users)
# Теперь проделываем то же самое с фильмами
new_movies = layers.Embedding(name="cf_xproducts_emb", input_dim=prd, output_dim=embeddings_size)(movies)
new_movies = layers.Reshape(name='cf_xproducts', target_shape=(embeddings_size,))(new_movies)
# Производим операцию Dot
cf_xx = layers.Dot(name='cf_xx', normalize=True, axes=1)([new_users, new_movies])

# Ниже приведем второй блок, который является полносвязным
# Также переводим в эмбеддинги и меняем размер для пользователей
second_user = layers.Embedding(name="nn_xusers_emb", input_dim=usr, output_dim=embeddings_size)(users)
second_user = layers.Reshape(name='nn_xusers', target_shape=(embeddings_size,))(second_user)
# Аналогично для фильмов
second_movie = layers.Embedding(name="nn_xproducts_emb", input_dim=prd, output_dim=embeddings_size)(movies)
second_movie = layers.Reshape(name='nn_xproducts', target_shape=(embeddings_size,))(second_movie)
# Соединяем данные пользователей и фильмов и применяем полносвязный слой 
nn_xx = layers.Concatenate()([second_user, second_movie])
nn_xx = layers.Dense(name="nn_xx", units=int(embeddings_size/2), activation='relu')(nn_xx)

# Третий блок работы с контекстом
# Считываем данные и используем полносвязный слой
contexts_in = layers.Input(name="contexts_in", shape=(ctx,))
contexts_in = layers.Dense(name="context_x", units=ctx, activation='relu')(contexts_in)

# Соединяем три блока
y_out = layers.Concatenate()([cf_xx, nn_xx, contexts_in])
y_out = layers.Dense(name="y_out", units=1, activation='linear')(y_out)
# Создаем модель с заданными параметрами
model = models.Model(inputs=[users,movies, contexts_in], outputs=y_out, name="Hybrid_Model")
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_percentage_error'])

ResourceExhaustedError: ignored

Если бы в нашей модели были бы еще данные о фильмах, то к данной сети добавился бы такой же блок, как и третий (считывание и полносвязный слой), но с другими данными

In [ ]:
# Обучение полученной модели с сохранением её весов каждую эпоху
EPOCHS = 10
filepath='/content/MAPE.hdf5'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=filepath,
    save_weights_only=True,
    monitor='val_mean_absolute_percentage_error',
    mode='max',
    save_best_only=True)
training = model.fit(x=[train['userId'], train['movieId'], train_context.values], y=train['rating'], epochs=EPOCHS, batch_size=128, shuffle=True, verbose=1, validation_split=0.3, callbacks=[model_checkpoint_callback])
model = training.model
# Тестировние модели
test['yhat'] = model.predict([test['userId'], test['movieId']])
test

Epoch 1/10


ValueError: ignored